## Set-up

In [1]:
!pip install pymysql

In [60]:
#Imports
import pandas as pd
import numpy as np
import datetime
import os
import warnings
import json
from sqlalchemy import create_engine

In [15]:
warnings.filterwarnings(action='ignore')

## Extraction

In [2]:
# Get the URL of the drive folder
folder_url = '../Data/Customer Loyalty'

In [4]:
# Get filenames within folder
all_files = os.listdir(folder_url)
trans_files = [string for string in all_files if string.startswith('trans')]
time_files = [string for string in all_files if string.startswith('time')]

In [5]:
# Import files one by one and append to a pandas dataframe
for i, file in enumerate(trans_files):
  # Create temporary string to access the file
  temp_string = folder_url + '/' + file
  # Import the temporary pandas dataframe for string
  temp_df = pd.read_csv(temp_string)
  # If this is the first one save it to a different variable else append it
  if i==0:
    trans_df = temp_df.copy()
  else:
    trans_df = trans_df.append(temp_df, ignore_index=True)
  # Debug break
  if i==-1:
    break

In [6]:
# Take a look at the dataframe
trans_df.head()

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200607,20060415,7,19,1,0.93,PRD0900033,CL00201,DEP00067,G00021,...,UM,OT,994100100398294,L,MM,Full Shop,Mixed,STORE00001,LS,E02
1,200607,20060413,5,20,1,1.03,PRD0900097,CL00001,DEP00001,G00001,...,LA,YF,994100100532898,L,LA,Top Up,Fresh,STORE00001,LS,E02
2,200607,20060416,1,14,1,0.98,PRD0900121,CL00063,DEP00019,G00007,...,NaN,NaN,994100100135562,L,MM,Top Up,Grocery,STORE00001,LS,E02
3,200607,20060415,7,19,1,3.07,PRD0900135,CL00201,DEP00067,G00021,...,UM,OT,994100100398294,L,MM,Full Shop,Mixed,STORE00001,LS,E02
4,200607,20060415,7,19,1,4.81,PRD0900220,CL00051,DEP00013,G00005,...,UM,OT,994100100398294,L,MM,Full Shop,Mixed,STORE00001,LS,E02


In [7]:
#Format the column names propperly
trans_df.columns = trans_df.columns.str.lower()

In [25]:
# Check the size
trans_df.shape

(3158469, 22)

Now we have a wide dataframe with all the files imported, however, this is inefficient, we will have to deal with this at some point. In this dataset, we have a set of 3158469 transactions. A transaction is recorded when a customer buys one SKU on a trip to any store, every set of items recorded for a trip is stored into a basket (or shopping cart). 

In [9]:
#Get now the time table
time_df =  pd.read_csv(folder_url + '/' + time_files[0])

## Transformation

At this transformation step we will take that wide dataframe aforementioned to distilled a series of tables that will be sent into the database described inside the `db_creation.sql` file. 

### Null values

In [10]:
# Review NAs for the whole dataframe
trans_df.isna().sum()

shop_week                        0
shop_date                        0
shop_weekday                     0
shop_hour                        0
quantity                         0
spend                            0
prod_code                        0
prod_code_10                     0
prod_code_20                     0
prod_code_30                     0
prod_code_40                     0
cust_code                   617450
cust_price_sensitivity      617450
cust_lifestage              924940
basket_id                        0
basket_size                      0
basket_price_sensitivity         0
basket_type                      0
basket_dominant_mission          0
store_code                       0
store_format                     0
store_region                     0
dtype: int64

We have 617450 transactions with unidentified customers and more than 300.000 transactions with identified customers without lifestage information. 

### Customers table

First, we are going to extract the customer data from the complete dataset in order to clean and send the information to the database.

In [30]:
# Extract the customer information
cust_df = trans_df[['cust_code', 'cust_price_sensitivity', 'cust_lifestage']]
cust_df.columns = ['cust_id', 'cust_price_sensitivity', 'cust_lifestage']

In [34]:
#Delete the rows without customer information and drop duplicates
cust_df.dropna(how='all', inplace=True)
cust_df.drop_duplicates(inplace=True)

In [35]:
#Review NAs
cust_df.isna().sum()

cust_id                     0
cust_price_sensitivity      0
cust_lifestage            521
dtype: int64

In [37]:
#Check the amount of identified customers without lifestage information
cust_df.cust_id[cust_df.cust_lifestage.isna()].nunique()

521

We have about 10% of the customers without lifestage information. Since the amount of unique rows for `cust_df` is equal to the amount of unique customers thart have lifestage set to `None`, we conclude that there's no existing information within the dataframe for the variable `cust_lifestage` to replace null values.

This information will be stored in the database after removing the duplicate entries into the `Customers` table.

In [36]:
# Take a look at the customers table
customers.head()

,cust_id,cust_price_sensitivity,cust_lifestage
0,CUST0000410727,UM,OT
1,CUST0000634693,LA,YF
47,CUST0000353957,LA,PE
50,CUST0000089820,LA,OT
52,CUST0000715467,MM,OT


We end up with a total of 5.000 customers to include.

## Timestamps table

The timestamps table contains all the information related to the time when the event (transaction) happens.

In [39]:
# Extract the time information
timestamps_df = trans_df[['shop_week', 'shop_date', 'shop_hour']].copy()
timestamps_df.drop_duplicates(inplace=True)
timestamps_df = timestamps_df.merge(time_df, on='shop_week', how='left', sort=True)
timestamps_df.reset_index(inplace=True)
timestamps_df.columns = ['time_id', 'shop_week', 'shop_date', 'shop_hour', 'date_from', 'date_to']

For the purpose of this analysis, dates will not be formatted, they will be stored as strings. In further steps this can easily be done and stored.

In [40]:
# And store the dataframe
timestamps = timestamps_df[['time_id','shop_week', 'date_from', 'date_to', 'shop_date', 'shop_hour']]

In [41]:
# Let's take a look at it.
timestamps.head()

,time_id,shop_week,date_from,date_to,shop_date,shop_hour
0,0,200607,20060410,20060416,20060415,19
1,1,200607,20060410,20060416,20060413,20
2,2,200607,20060410,20060416,20060416,14
3,3,200607,20060410,20060416,20060412,19
4,4,200607,20060410,20060416,20060413,18


## Products table

The products table contains the product code and the categories to which each product fits inside the product hierarchy. At the time no product name/brand is available. 

In [42]:
# Extract the products information
products_df = trans_df[['prod_code', 'prod_code_10', 'prod_code_20', 'prod_code_30', 'prod_code_40']].copy()
products_df.columns = ['prod_id', 'prod_code_10', 'prod_code_20', 'prod_code_30', 'prod_code_40']
products_df.drop_duplicates(inplace=True)

In [43]:
products_df.head()

,prod_id,prod_code_10,prod_code_20,prod_code_30,prod_code_40
0,PRD0900033,CL00201,DEP00067,G00021,D00005
1,PRD0900097,CL00001,DEP00001,G00001,D00001
2,PRD0900121,CL00063,DEP00019,G00007,D00002
3,PRD0900135,CL00201,DEP00067,G00021,D00005
4,PRD0900220,CL00051,DEP00013,G00005,D00002


In [44]:
# And store the dataframe
products = products_df

## Baskets table

The baskets table will have all the information related to the baskets to which the products belong.

In [45]:
# Extract the baskets information
baskets_df = trans_df[['basket_id', 'basket_size', 'basket_price_sensitivity', 'basket_type', 'basket_dominant_mission']].copy()
baskets_df.drop_duplicates(inplace=True)

In [46]:
baskets_df.head()

,basket_id,basket_size,basket_price_sensitivity,basket_type,basket_dominant_mission
0,994100100398294,L,MM,Full Shop,Mixed
1,994100100532898,L,LA,Top Up,Fresh
2,994100100135562,L,MM,Top Up,Grocery
5,994100100532897,M,MM,Small Shop,Fresh
6,994100100136577,M,LA,Small Shop,Fresh


In [47]:
# And store the dataframe
baskets = baskets_df

## Stores table

This table contains the stores at which the transaction was registered along with some other information. 

In [48]:
# Extract the stores information
stores_df = trans_df[['store_code', 'store_format', 'store_region']].copy()
stores_df.columns = ['store_id', 'store_format', 'store_region']
stores_df.drop_duplicates(inplace=True)

In [49]:
stores_df.head()

,store_id,store_format,store_region
0,STORE00001,LS,E02
47,STORE00002,LS,W01
94,STORE00003,LS,E01
145,STORE00004,MS,E03
155,STORE00006,LS,S01


In [50]:
# And store the dataframe
stores = stores_df

## Transactions table

Finally, the transactions table will be our main table where each event is recorded and all foreign tables are cited. 

In [51]:
# Extract the stores information
transactions_df = trans_df[['shop_week', 'shop_date', 'shop_hour',
                            'quantity', 'spend', 'prod_code', 'cust_code','basket_id', 'store_code']]
transactions_df.columns = ['shop_week', 'shop_date','shop_hour',
                           'quantity', 'spend', 'prod_id', 'cust_id','basket_id', 'store_id']

In [52]:
# Append the time_id
transactions_df = transactions_df.merge(timestamps[['shop_date', 'shop_hour', 'time_id']], 
                                  how='left',  
                                  on=['shop_date', 'shop_hour'])
transactions_df.drop(columns=['shop_week', 'shop_date', 'shop_hour'], inplace=True)

In [53]:
transactions = transactions_df[['time_id','quantity', 'spend', 'prod_id', 'cust_id', 'basket_id', 'store_id']]

In [54]:
transactions.head()

,time_id,quantity,spend,prod_id,cust_id,basket_id,store_id
0,0,1,0.93,PRD0900033,CUST0000410727,994100100398294,STORE00001
1,1,1,1.03,PRD0900097,CUST0000634693,994100100532898,STORE00001
2,2,1,0.98,PRD0900121,NaN,994100100135562,STORE00001
3,0,1,3.07,PRD0900135,CUST0000410727,994100100398294,STORE00001
4,0,1,4.81,PRD0900220,CUST0000410727,994100100398294,STORE00001


# Loading data

This step creates the connection to the database and loops through all the tables inside the database populating it with the data created inside this script. 

In [62]:
# Load the file 
json_data = json.loads(open('../../db_conn.json').read())

conn = 'mysql+pymysql://'+json_data["user"]+':'+json_data["password"]+'@'+json_data["host"]+':'+str(json_data["port"])+'/'+json_data["db_name"]

In [65]:
# Connect to the DB
mysql_engine = create_engine(conn,echo=False)

In [66]:
# Set tables names to iterate
sql_tables = mysql_engine.table_names()
pd_tables = [tab.lower() for tab in sql_tables]

In [67]:
sql_tables

['Baskets', 'Customers', 'Products', 'Stores', 'Timestamps', 'Transactions']

In [194]:
#Iterate over database tables and send to sql query database
for pdtab, sqltab in zip(pd_tables, sql_tables):
  print(pdtab, sqltab, eval(pdtab).shape[0])
  eval(pdtab).to_sql(sqltab, mysql_engine, if_exists='append', index=False)

baskets Baskets 490982
customers Customers 5000
products Products 4997
stores Stores 761
timestamps Timestamps 11467
transactions Transactions 3158469


In [69]:
mysql_engine.dispose()

At the end of this script a database has been created with all the data provided, which is a more efficient way to store and query the information. 